## Data Loading

Using the quora question duplicates dataset. Loads question pairs into train/test_question_pairs and whether the pair is a duplicate into train/test_question_labels.

In [ ]:
quora_duplicates_train_filename = "data/train.csv"

In [ ]:
import csv # for parsing data
import matplotlib.pyplot as plt # for plotting results
# so that graphs show up in the notebook
%matplotlib inline

In [ ]:
import sklearn # Common machine learning functions

In [ ]:
def read_question_input_file(filename):
    pairs_list = []
    labels_list = []
    with open(filename) as f:
        f.readline() # consume the csv top row which has column names
        for row in csv.reader(f):
            pairs_list.append((row[3], row[4])) # The two questions for each row
            labels_list.append(int(row[5])) # this is whether the questions were marked as duplicates
    return pairs_list, labels_list
            
question_pairs, question_labels = read_question_input_file(quora_duplicates_train_filename)

# Shuffle the input together to ensure random split between train and test
# If you forget to do this and your data is ordered, you'll often see much higher train than test accuracy
question_pairs, question_labels = sklearn.utils.shuffle(question_pairs, question_labels)

In [ ]:
# Split into train and test data
# Will be used later, initially just working with train
train_cutoff = int(len(question_pairs)*0.7)
train_question_pairs = question_pairs[:train_cutoff]
train_question_labels = question_labels[:train_cutoff]

test_question_pairs = question_pairs[train_cutoff:]
test_question_labels = question_labels[train_cutoff:]


all_train_questions = []
all_test_questions = []
for pair in train_question_pairs:
    all_train_questions.extend([pair[0], pair[1]])
for pair in test_question_pairs:
    all_test_questions.extend([pair[0], pair[1]])

In [ ]:
print("num question pairs: ", len(question_pairs))
print("num train question pairs: ", len(train_question_pairs))
print("num test question pairs: ", len(test_question_pairs))
assert(len(question_pairs) == len(question_labels))

## NLTK setup

Some Additional datasets are needed for the operations we're using in nltk

In [ ]:
import nltk
import string
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

# Text Processing

The code below is part of the presentation on text processing.

### Text Preprocessing

`preprocess_question` is a function to perform various text normalization techniques. Primarily, this is done by attempting to reduce the text to the simplest form that still carries all the meaning that we're concerned with.

In [ ]:
lemmatizer = nltk.stem.WordNetLemmatizer()
stopwords = set(nltk.corpus.stopwords.words("english"))
stemmer = nltk.stem.PorterStemmer()

def preprocess_question(question,
                        split_method="spaces", # either "spaces" or "tokenization"
                        use_lowercase=False,
                        stem_method=None, # None, "stemming" or "lemmatize"
                        use_remove_stopwords=False,
                        use_remove_punctuation=False,
                        verbose=False):
    """Takes as input a question text string, and produces a list of tokens.
    
    split_method: either "spaces" or "tokenization". Determines method used to split up string.
    use_lowercase: if True, will lower case all tokens
    stem_method: None, "stemming" or "lemmatize". Determines method used to find base word.
    use_remove_stopwords: If True, will remove stopwords from the question tokens
    use_remove_punctuation: If True, will remove all punctuation tokens
    verbose: If True, will print the results of each step.
    
    """
    
    def vprint(*args):
        if verbose:
            print(args)

    vprint("input question:", question)
    
    def tokenize(question):
        # Take the text and break it into words
        # Handles punctuation better than text.split()
        tokens = nltk.word_tokenize(question)
        vprint("tokenized:", tokens)
        return tokens
    
    def basic_split(question):
        tokens = question.split(' ')
        vprint("split:", tokens)
        return tokens

    def lowercase(tokens):
        tokens = [t.lower() for t in tokens] 
        return tokens
    
    def stem(tokens):
        # For non-acrynonyms
        tokens = [stemmer.stem(t) for t in tokens]
        vprint("stemmed:", tokens)
        return tokens
    
    def lemmatize(tokens):
        token_pos_pairs = nltk.pos_tag(tokens)
        vprint("part of speech tagged: ", token_pos_pairs)
        tokens = [lemmatizer.lemmatize(pair[0], get_wordnet_pos(pair[1])) for pair in token_pos_pairs]
        vprint("lemmatized:", tokens)
        return tokens
    
    def remove_stopwords(tokens):
        tokens = [t for t in tokens if t not in stopwords]
        vprint("stopwords removed:", tokens)
        return tokens
    
    def remove_punctuation(tokens):
        tokens = [t for t in tokens if t not in string.punctuation]
        vprint('punctuation removed', tokens)
        return tokens
    
    if split_method == "spaces":
        tokens = basic_split(question)
    else:
        tokens = tokenize(question)
    
    if use_lowercase:
        tokens = lowercase(tokens)
    
    if stem_method == "stemming":
        tokens = stem(tokens)
    elif stem_method == "lemmatize":
        tokens = lemmatize(tokens)
    
    if use_remove_stopwords:
        tokens = remove_stopwords(tokens)
        
    if use_remove_punctuation:
        tokens = remove_punctuation(tokens)
    
    return tokens

test_tokens = preprocess_question(
    "In tennis, why isn't 30-30 called Deuce? And 40-30 (or 30-40) called Advantage?",
    verbose=True)

### Predicting similarty with simple similarity methods

Starting with the simple operation of returning whether the number of shared tokens after preprocessing are greater than some threshold.

In [ ]:
def basic_predictor(q1, q2, threshold=3, **kwargs):
    """Takes two questions, returns true if the intersection of their tokens is larger than threshold."""
    q1_tokens = set(preprocess_question(q1, **kwargs))
    q2_tokens = set(preprocess_question(q2, **kwargs))
    return len(q1_tokens.intersection(q2_tokens)) >= threshold    

In [ ]:
def eval_predictor_binary(predictor, question_pairs, labels, **kwargs):
    """Given a predictor function and data to evaluate on, calculate recall, precision and accuracy."""
    y_pred = []
    for pair in question_pairs:
        y_pred.append(int(predictor(*pair, **kwargs)))
    
    # Of all labels that should have been predicted positive, how many were?
    recall = sklearn.metrics.recall_score(y_true=labels,
                                          y_pred=y_pred)
    print('recall:', recall)
    
    # Of the labels that were predicted positive, how many actually should have been?
    precision = sklearn.metrics.precision_score(y_true=labels,
                                                y_pred=y_pred)
    print('precision:', precision) 
    
    # What percent of predictions matched the true label?
    accuracy = sklearn.metrics.accuracy_score(y_true=labels,
                                              y_pred=y_pred)
    print('accuracy:', accuracy)

Trying out various configurations of preprocessing and thresholds:

In [ ]:
cutoff = 1000 # This takes awhile otherwise, use a smaller value if this takes too long for you
eval_pairs = train_question_pairs[:cutoff]
eval_labels = train_question_labels[:cutoff]

basic_args = (basic_predictor,
              eval_pairs,
              eval_labels)

for threshold in [2,5]:
    print('\nTHRESHOLD', threshold)
    eval_predictor_binary(*basic_args,
                          threshold=threshold)
    print('TOKENIZATION')
    eval_predictor_binary(*basic_args,
                           threshold=threshold,
                           split_method="tokenization")
    print('LOWERCASE')
    eval_predictor_binary(*basic_args,
                           threshold=threshold,
                           split_method="tokenization",
                           use_lowercase=True)
    print('ALL')
    eval_predictor_binary(*basic_args,
                           threshold=threshold,
                           split_method="tokenization",
                           use_lowercase=True,
                           stem_method="lemmatize",
                           use_remove_stopwords=True,
                           use_remove_punctuation=True)

### Baseline comparison

Compare to the predictor that simply returns a random result to get a baseline.

In [ ]:
import random
def random_predictor(q1, q2, prob_pos=0.5):
    return random.random() < prob_pos

In [ ]:
# baseline
random_args = (random_predictor, eval_pairs, eval_labels)
print('RANDOM BASELINE')
eval_predictor_binary(*random_args, prob_pos = 0.5)
print('20% positive')
eval_predictor_binary(*random_args, prob_pos = 0.2)
print('1% positive')
eval_predictor_binary(*random_args, prob_pos = 0.01)

### Probability predictions

Instead of using a fixed threshold, output a real number, and then a threshold can be used later.

In [ ]:
def basic_prob_predictor(q1, q2, use_jaccard=False, **kwargs):
    """Basic real valued predictoin using jaccard similarity, or a simple measure of intersection/10."""
    q1_tokens = set(preprocess_question(q1, **kwargs))
    q2_tokens = set(preprocess_question(q2, **kwargs))
    q_intersect = len(q1_tokens.intersection(q2_tokens))
    if use_jaccard:
        q_union = 1 + len(q1_tokens.union(q2_tokens))
        return q_intersect/q_union
    else:
        return q_intersect/10

In [ ]:
def random_predictor_prob(q1, q2):
    """Baaseline random probability comparison"""
    return random.random()

### Probability predictor evaluation

Here, we use pyplot to graphically show how well the classifier predicts.

There's a tradeoff between precision and recall. If we simply return that all pairs are duplicates, the recall will be 1.0. On the other hand, if we only mark exact text matches as duplicates, our precision will be 1.0, but our recall will be extremely low. An F1-score is an attempt at balancing the importance of precision and recall, but it is insufficient to look only at the max F1-score to determine the best classifier. Some use cases may care much more about precision than recall, or vise-versa.


In [ ]:
def eval_predictor_prob(predictor, question_pairs, labels, name="", **kwargs):
    pred = []
    for pair in question_pairs:
        pred.append(predictor(*pair, **kwargs))
    
    precision, recall, threshold = sklearn.metrics.precision_recall_curve(
                                        y_true=labels,
                                        probas_pred=pred)
    
    
    # f1_score is the harmonic average between precision and recall
    f1_score = [2 * (precision[i] * recall[i]) 
                / (precision[i] + recall[i]) for i in range(len(precision))]
    
    fig, plts = plt.subplots(1, 3, figsize=[12,4])
    fig.suptitle(name)
    fig.subplots_adjust(hspace=0.5)
    
    for subplot in plts:
        subplot.grid(True)
        subplot.set_ylim(ymin=0, ymax=1)
    
    plts[0].set(ylabel="Recall/Precision")
    plts[0].set(xlabel="Threshold")
    plts[0].grid(True)
    plts[0].plot(threshold, precision[:-1], 'b', label="precision")
    plts[0].plot(threshold, recall[:-1], 'r', label="recall")
    plts[0].legend()
    
    plts[1].set(ylabel="Precision")
    plts[1].set(xlabel="Recall")
    plts[1].grid(True)
    plts[1].plot(recall, precision)
    
    plts[2].set(ylabel="F1 Score")
    plts[2].set(xlabel="Threshold")
    plts[2].grid(True)
    plts[2].plot(threshold, f1_score[:-1])

In [ ]:
eval_predictor_prob(random_predictor_prob, eval_pairs, eval_labels, name="Random Probability")

In [ ]:
eval_predictor_prob(basic_prob_predictor, eval_pairs, eval_labels,
                    name="Space Split Jaccard",
                    use_jaccard=True)
eval_predictor_prob(basic_prob_predictor, eval_pairs, eval_labels,
                    name="All normalization Jaccard",
                    use_jaccard=True,
                    split_method="tokenization",
                    use_lowercase=True,
                    stem_method="lemmatize",
                    use_remove_stopwords=True,
                    use_remove_punctuation=True)

### N Grams

This is a common technique to take into account some degree of word ordering when using bag of word techniques. Instead of only considering single words (unigrams), we also consider the combination of `n` consecutive tokens.



In [ ]:
def produce_word_n_grams(tokens, n_values=[1,2]):
    """Given a list of tokens and n_values, produce a list of corresponding word n_grams
    
    For example, if n_values = [1], this will produce only unigrams.
    If n_values = [1,2], this will proiduce unigrams and bigrams.
    """
    ret = []
    for n in n_values:
        for i in range(len(tokens) + 1 - n):
            ret.append(" ".join(tokens[i+k] for k in range(n)))
    return ret

print(produce_word_n_grams(test_tokens))

In [ ]:
def n_gram_sim_predictor(q1, q2, n_values=[1,2], **kwargs):
    q1_tokens = preprocess_question(q1, **kwargs)
    q2_tokens = preprocess_question(q2, **kwargs)
    q1_ngrams = set(produce_word_n_grams(q1_tokens, n_values))
    q2_ngrams = set(produce_word_n_grams(q2_tokens, n_values))
    q_intersect = len(q1_ngrams.intersection(q2_ngrams))
    q_union = 1 + len(q1_ngrams.union(q2_ngrams)) # 1 + to handle divide by 0
    return q_intersect/q_union # jaccard similarity

In [ ]:
eval_predictor_prob(n_gram_sim_predictor, eval_pairs, eval_labels,
                    name="n-gram predictor unigram",
                    n_values=[1],
                    split_method="tokenization",
                    use_lowercase=True,
                    stem_method=None,
                    use_remove_stopwords=True,
                    use_remove_punctuation=True)
eval_predictor_prob(n_gram_sim_predictor, eval_pairs, eval_labels,
                    name="n-gram predictor bigram",
                    n_values=[2],
                    split_method="tokenization",
                    use_lowercase=True,
                    stem_method=None,
                    use_remove_stopwords=True,
                    use_remove_punctuation=True)


In [ ]:
# this one l2 normalizes the produced vectors
tfidf_vectorizer_norm = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,1))
tfidf_vectorizer_norm.fit(all_train_questions)

# this one does not
tfidf_vectorizer = sklearn.feature_extraction.text.TfidfVectorizer(ngram_range=(1,1), norm=None)
tfidf_vectorizer.fit(all_train_questions)

# Both do lowercasing, tokenizing, removes punctuation already for us
# Can also produce ngrams

In [ ]:
def tfidf_cos_sim(q1, q2):
    """Computes the cos similiarity of two """
    # produces two l2 normalized vectors
    tfidf_vectors = tfidf_vectorizer_norm.transform([q1,q2])

    # This is a dot product, but same as cos-sim since vectors are normalized
    return tfidf_vectors.getrow(0).multiply(tfidf_vectors.getrow(1)).sum()

# This function will lowercase, tokenize, remove punct, make ngrams
tfidf_analyzer = tfidf_vectorizer.build_analyzer()

def get_idf_sum(tokens):
    return sum([tfidf_vectorizer.idf_[tfidf_vectorizer.vocabulary_[t]] for t in tokens])

def idf_jaccard_sim(q1, q2):
    """For questions, it doesn't make much sense to use the tf, so just sum idf"""
    q1_tokens = set(tfidf_analyzer(q1))
    q2_tokens = set(tfidf_analyzer(q2))
    
    q_intersect = q1_tokens.intersection(q2_tokens)
    q_union = q1_tokens.union(q2_tokens)
    
    return get_idf_sum(q_intersect)/get_idf_sum(q_union)

def idf_constant_sim(q1, q2):
    """For questions, it doesn't make much sense to use the tf, so just sum idf"""
    q1_tokens = set(tfidf_analyzer(q1))
    q2_tokens = set(tfidf_analyzer(q2))
    
    q_intersect = q1_tokens.intersection(q2_tokens)
    
    return get_idf_sum(q_intersect)/100

In [ ]:
# This cell takes a long time to run
eval_predictor_prob(idf_jaccard_sim, eval_pairs[:1000], eval_labels[:1000], name="jaccard idf")
eval_predictor_prob(tfidf_cos_sim, eval_pairs, eval_labels, name="cos sim tfidf")